### libraries

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
print(torch.__version__)

device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# configuration
block_size = 8  # how many characters in context 
batch_size = 64   # how many sampler or block size that use in parallel compute 
max_iters = 1000
# eval_internal = 2500
learning_rate = 0.0004
eval_internal = 250

2.0.1+cu117
cuda


###  Convert words in text file to chars list 

In [2]:
with open('data/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# put words from .txt into char list
chars = sorted(set(text))
vocab_size = len(chars)
print("Size of vocab: ", vocab_size)
print("List of chars: ",chars)

Size of vocab:  81
List of chars:  ['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### encode decode function

In [3]:
# create dic to map chars to int from 0 - len of chars
import torch
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}


# create encode function
encode = lambda s: [string_to_int[c] for c in s]

def encode2(s):
  encoded_list = []
  for c in s:
    encoded_list.append(string_to_int[c])
  return encoded_list
    
    
 # create decode function  
decode = lambda l: ''.join([int_to_string[i] for i in l])

def decode2(l):
  decoded_string = ''
  for i in l:
    decoded_string += int_to_string[i]
  return decoded_string


# test encode decode func 
encoded_word = encode2("llm from sratch")
print(encoded_word)

decoded_word = decode2(encoded_word)
print(decoded_word)

print('\n------------------ tensor dtype ------------------\n')

# test encode decode func with tensor dtype
encoded_word2 = torch.tensor(encode2("llm from sratch"), dtype=torch.long)
print(encoded_word2)
print(type(encoded_word2))

# decoded_word2 = decode(encoded_word2)
# print(decoded_word2)



[65, 65, 66, 1, 59, 71, 68, 66, 1, 72, 71, 54, 73, 56, 61]
llm from sratch

------------------ tensor dtype ------------------

tensor([65, 65, 66,  1, 59, 71, 68, 66,  1, 72, 71, 54, 73, 56, 61])
<class 'torch.Tensor'>


### using torch

In [4]:
# convert data from plaint text to tensor dtype 
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:50])
print(len(data))

# split data to train/test
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]



tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1])
232309


In [7]:
# batch size hyperparameters 

block_size = 8

x = train_data[:block_size]
y = train_data[block_size:]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "the target is", target)
    

when input is tensor([80]) the target is tensor(32)
when input is tensor([80,  1]) the target is tensor(49)
when input is tensor([80,  1,  1]) the target is tensor(1)
when input is tensor([80,  1,  1, 28]) the target is tensor(25)
when input is tensor([80,  1,  1, 28, 39]) the target is tensor(38)
when input is tensor([80,  1,  1, 28, 39, 42]) the target is tensor(28)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) the target is tensor(1)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) the target is tensor(44)
